In [1]:
import pandas as pd

In [2]:
restoran_df = pd.read_csv('../data/dataFinal.csv')

In [3]:
restoran_df = restoran_df.drop(['Unnamed: 0','Price'],axis=1)

In [4]:
restoran_df = restoran_df.drop(index=135, axis=0)

In [5]:
restoran_df.head()

,Restaurant_ID,Name,Type,Area,Rating,Review,Restaurant Address,Latitude,Longitude
0,0,Hause Rooftop,"['Casual Dining', 'Bar', 'Western', 'Coffee', ...",Setiabudi,4.3,1282.0,"MD Place Bulding, Lantai 6, Jl. Setiabudi Sela...",-62.081.643.570,106.828.454.285
1,1,The Neighbourhood,"['Fine Dining', 'Indonesian']",Dharmawangsa,4.4,770.0,"Jl. Cipaku I No. 85, Dharmawangsa, Jakarta 12170",-62.395.528.820,106.814.613.081
2,2,SKYE,"['Fine Dining', 'Lounge', 'Fusion', 'European'...",Thamrin,4.3,2827.0,"Menara BCA, Lantai 56, Jl. MH. Thamrin, Thamri...",-61.967.130.000,106.822.580.000
3,3,WAKI Japanese BBQ Dining,"['Casual Dining', 'Japanese', 'BBQ']",Thamrin,4.9,2995.0,"Lantai 1, Jl. Tanjung Karang No. 5, Thamrin, J...",-62.011.260.000,106.822.306.000
4,4,3 Wise Monkeys,"['Casual Dining', 'Japanese', 'Sushi']",Senopati,4.2,2091.0,"Jl. Senopati No. 46, Senopati, Jakarta",-62.311.835.698,106.808.081.232


# Weighted Filtering

In [6]:
def calculate(restoran_df, restoran_type, restoran_area, prioritas_1, prioritas_2, prioritas_3):
    value_df = []

    type_prio = (0.5 if prioritas_1 == "jenis" else 0.3 if prioritas_2 == "jenis" else 0.2)
    area_prio = (0.5 if prioritas_1 == "lokasi" else 0.3 if prioritas_2 == "lokasi" else 0.2)
    rating_prio = (0.5 if prioritas_1 == "rating" else 0.3 if prioritas_2 == "rating" else 0.2)

    for x in range(len(restoran_df)):
        resto_value = 0

        if restoran_type in str(restoran_df[restoran_df['Restaurant_ID'] == x]['Type'].values[0]):
            resto_value += type_prio * 1

        if restoran_area == str(restoran_df[restoran_df['Restaurant_ID'] == x]['Area'].values[0]):
            resto_value += area_prio * 1

        resto_value += rating_prio * (restoran_df[restoran_df['Restaurant_ID'] == x]['Rating'].values[0]/5)

        value_df.append(resto_value)

    return value_df

In [17]:
value_df = calculate(restoran_df, "Casual Dining", "Setiabudi", "rating", "jenis", "lokasi")

In [18]:
restoran_df = restoran_df.sort_values(by='Restaurant_ID')
restoran_df['Value'] = value_df

In [19]:
restoran_df = restoran_df.sort_values(by='Value', ascending=False)

In [20]:
print('The 3 Most recommended restaurants for You are:\n')
j = 0
for x in range(3):
    print(j+1, restoran_df['Name'].values[x])
    j = j + 1

The 3 Most recommended restaurants for You are:

1 Pokenbir
2 Warung MJS
3 Hause Rooftop


# Content Based Filtering

In [11]:
import re as re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
def get_important_features(data):
    important_features = []
    for i in range(0, data.shape[0]):
        x = (str(data['Type'][i])+' '+str(data['Area'][i]))
        x = x.lower()
        x = re.sub(r'[^.,a-zA-Z0-9 \n\.]',' ',x)
        x = re.sub(r'[^\w\s]','',x)
        x = re.sub('[\s]+', ' ', x)
        important_features.append(x)

    return important_features

In [13]:
restoran_df['Important_Features'] = get_important_features(restoran_df)

In [14]:
cm = CountVectorizer().fit_transform(restoran_df['Important_Features'])
cs = cosine_similarity(cm)

In [15]:
product_name = 'OKU - Hotel Indonesia Kempinski'

product_id = restoran_df[restoran_df["Name"] == product_name]['Restaurant_ID'].values[0]

scores = list(enumerate(cs[product_id]))

sorted_scores = sorted(scores, key = lambda x:x[1], reverse=True)
sorted_scores = sorted_scores[1:4]

In [16]:
print('The 3 Most recommended restaurants to', product_name, 'are:\n')
j = 0
for item in sorted_scores:
    product_name = restoran_df[restoran_df['Restaurant_ID'] == sorted_scores[j][0]]['Name'].values[0]
    print(j+1, product_name)
    j = j + 1

The 3 Most recommended restaurants to OKU - Hotel Indonesia Kempinski are:

1 Henshin - The Westin Jakarta
2 Sana Sini Restaurant - Pullman Jakarta Indonesia
3 OSO Ristorante
